In [604]:
# Library
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import DictionaryLearning, sparse_encode
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

# Load data
# Load data
# measurement = np.load('../test/dataset/meas_symm_1.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_symm_2.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_symm_3.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_symm_4.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_symm_5.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_symm_varspeed_1.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_asymm_1.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_asymm_2.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_asymm_reflector_1.npz', allow_pickle=False)
measurement = np.load('../test/dataset/meas_asymm_reflector_2.npz', allow_pickle=False)

header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:1000]  # Using 1000 samples for simplicity

trainCIR, tempCIR = train_test_split(data_cir, test_size=0.4, random_state=42)

# Now, split tempCIR into test (20%) and evaluation (20%) sets
testCIR, evalCIR = train_test_split(tempCIR, test_size=0.5, random_state=42)

print(f"Train Data: {trainCIR.shape}")
print(f"Test Data: {testCIR.shape}")
print(f"Evaluation Data: {evalCIR.shape}")

# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6    # Channel 6 is EVE (illegitimate)

# Set the number of dictionary atoms (components) and sparsity level
# N_components = 3  # Number of Sparse Components

n_components = 5 
n_components = 10
n_components = 15
n_components = 20
# n_nonzero_coefs = 30 # Sparsity level

Train Data: (600, 15, 251, 2)
Test Data: (200, 15, 251, 2)
Evaluation Data: (200, 15, 251, 2)


In [605]:
# ----------------------------------------------------- Preprocessing -----------------------------------------------------
# ----------------- Training data -----------------
train_alice_cirs = trainCIR[:, alice_channel, :, :]  # Alice's CIRs
train_eve_cirs = trainCIR[:, eve_channel, :, :]      # Eve's CIRs
train_alice_magnitude = np.abs(train_alice_cirs[..., 0] + 1j * train_alice_cirs[..., 1])
train_eve_magnitude = np.abs(train_eve_cirs[..., 0] + 1j * train_eve_cirs[..., 1])
train_data_combined  = np.vstack((train_alice_magnitude, train_eve_magnitude))

# Learn the dictionary
dict_learner = DictionaryLearning(n_components=n_components, transform_algorithm='lasso_lars', n_jobs= -1)
# dict_learner = DictionaryLearning(n_components=n_components, transform_algorithm='omp', transform_n_nonzero_coefs=n_nonzero_coefs,  n_jobs= -1)
dict_learner.fit(train_data_combined)
D = dict_learner.components_  # Shape: (n_components, feature_dim)
print(D.shape)

train_alice_sparse_codes = dict_learner.transform(train_alice_magnitude)
train_eve_sparse_codes = dict_learner.transform(train_eve_magnitude)
train_sparse_codes = np.vstack((train_alice_sparse_codes, train_eve_sparse_codes))
# print(train_alice_sparse_codes.shape)
print(train_alice_sparse_codes[0])

(20, 251)
[44563.836674    6678.21543716  1459.85407267 -3552.94847113
 -5416.00109494  1357.92339736  2812.1617227  13351.5434469
 -5547.6442559   1667.63009017  1251.14012031  -439.58181313
  4304.26649799   640.04911611 -1174.39239528  6188.34186048
  3176.84007738   484.98325108  -351.82529531 -3919.6115697 ]


In [606]:
test_alice_CIRs = testCIR[:, alice_channel, :, :]
test_eve_CIRs = testCIR[:, eve_channel, :, :]

# test amplitude
test_alice_magnitude = np.abs(test_alice_CIRs[..., 0] + 1j * test_alice_CIRs[..., 1])
test_eve_magnitude = np.abs(test_eve_CIRs[..., 0] + 1j * test_eve_CIRs[..., 1]) 
test_cirs = np.vstack((test_alice_magnitude, test_eve_magnitude))


# Transform test data into sparse codes
test_alice_sparse_codes = dict_learner.transform(test_alice_magnitude)
test_eve_sparse_codes = dict_learner.transform(test_eve_magnitude)

# Combine the sparse codes
test_sparse_codes = np.vstack((test_alice_sparse_codes, test_eve_sparse_codes))
print('Test data shape:', test_sparse_codes.shape)

Test data shape: (400, 20)


In [607]:
# Labels for training data
train_alice_labels = np.zeros(train_alice_sparse_codes.shape[0])  # Label '0' for Alice.
train_eve_labels = np.ones(train_eve_sparse_codes.shape[0])       # Label '1' for Eve.
train_labels = np.hstack((train_alice_labels, train_eve_labels))

# Labels for test data
test_alice_labels = np.zeros(test_alice_sparse_codes.shape[0])  # Label '0' for Alice.
test_eve_labels = np.ones(test_eve_sparse_codes.shape[0])       # Label '1' for Eve.
test_labels = np.hstack((test_alice_labels, test_eve_labels))


In [608]:
# # Dictionary
# D = train_sparse_codes.T
# print('Dictionary shape:', D.shape)

In [609]:
scaler = StandardScaler()
train_sparse_codes = scaler.fit_transform(train_sparse_codes)
# Apply the same scaling to test data
test_sparse_codes = scaler.transform(test_sparse_codes)
print('Train data shape:', train_sparse_codes.shape)

Train data shape: (1200, 20)


In [610]:
# Initialize and train the SVM classifier
classifier = SVC(kernel='rbf', random_state=42)
classifier.fit(train_sparse_codes, train_labels)

# Predict on test data
predictions = classifier.predict(test_sparse_codes)

In [611]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Calculate MDR, FAR, AR
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp)) if ((tp + fn) + gamma * (tn + fp)) > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


Classification Accuracy: 78.00%
tp: 153
tn: 159
fp: 41
fn: 47
MDR: 0.205
FAR: 0.235
AR: 0.78


In [612]:
# -------- Evaluation --------
# Extract CIRs
eval_alice_CIRs = evalCIR[:, alice_channel, :, :]
eval_eve_CIRs = evalCIR[:, eve_channel, :, :]

# Compute magnitudes
eval_alice_magnitude = np.abs(eval_alice_CIRs[..., 0] + 1j * eval_alice_CIRs[..., 1])
eval_eve_magnitude = np.abs(eval_eve_CIRs[..., 0] + 1j * eval_eve_CIRs[..., 1])

# Stack evaluation features
eval_cirs = np.vstack((eval_alice_magnitude, eval_eve_magnitude))

# Transform evaluation data into sparse codes
eval_alice_sparse_codes = dict_learner.transform(eval_alice_magnitude)
eval_eve_sparse_codes = dict_learner.transform(eval_eve_magnitude)

# Combine the sparse representations
eval_sparse_codes = np.vstack((eval_alice_sparse_codes, eval_eve_sparse_codes))
# print('Evaluation data shape:', eval_sparse_codes.shape)

# Scale evaluation data using the trained scaler
eval_sparse_codes = scaler.transform(eval_sparse_codes)

# Labels for evaluation data
eval_alice_labels = np.zeros(eval_alice_sparse_codes.shape[0])  # Label '0' for Alice
eval_eve_labels = np.ones(eval_eve_sparse_codes.shape[0])       # Label '1' for Eve
eval_labels = np.hstack((eval_alice_labels, eval_eve_labels))

# Predict on evaluation data
eval_predictions = classifier.predict(eval_sparse_codes)

eval_accuracy = accuracy_score(eval_labels, eval_predictions)
print(f"Classification Accuracy: {eval_accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(eval_labels, eval_predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Calculate MDR, FAR, AR
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp)) if ((tp + fn) + gamma * (tn + fp)) > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")



Classification Accuracy: 75.00%
tp: 142
tn: 158
fp: 42
fn: 58
MDR: 0.21
FAR: 0.29
AR: 0.75
